# Doodle classifier


In [5]:
# Setup
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import os
import sys
root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

import numpy as np
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from src.data_loader import DataLoader as DL

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Training

In [8]:
dl = DL()
data, labels = dl.load_data_batches(batch_size=10_000)
data_1d = np.array([dl.one_dimensional_array_from_matrix(mat) for mat in data])
X_train, X_test, y_train, y_test = train_test_split(data_1d, labels, test_size=0.3, shuffle=True)

In [9]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

21000 9000 21000 9000


In [10]:
# hidden_layer_sizes=(128, 128, 64)
clf = MLPClassifier(hidden_layer_sizes=(128,))
clf.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(128,))

## Prediction

In [5]:
clf.score(X_test, y_test)

0.7733333333333333

In [1]:
pred = clf.predict(X_test)
confusion_matrix(y_test, pred)

NameError: name 'clf' is not defined

### Results

| Parameter | Value | Results (Accuray) |
| --- | --- | --- |
| Hidden Layers | `(100,)` (default) | `0.811` |
| Hidden Layers | `(128, 128, 64)` | `0.868` |
| Hidden Layers | `(128,)` | `0.863` |


